In [ ]:
!pip -q install ultralytics==8.3.29 pycocotools==2.0.7 opencv-python-headless==4.10.0.84

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

BASE = "/content/drive/MyDrive/Archive"
IMAGES_DIR = f"{BASE}/images"                 # your images
COCO_JSON  = f"{BASE}/result_modified.json"   # your COCO (ids start at 1)
DATA_ROOT  = f"{BASE}/yolo"                   # new YOLO dataset root we create


Mounted at /content/drive


In [ ]:
import json, random, os
os.makedirs(DATA_ROOT, exist_ok=True)

SPLIT_SEED = 42
VAL_FRAC   = 0.2

with open(COCO_JSON, "r") as f:
    coco = json.load(f)

img_ids = [im["id"] for im in coco["images"]]
random.Random(SPLIT_SEED).shuffle(img_ids)
n_val = max(1, int(len(img_ids) * VAL_FRAC))
val_ids   = set(img_ids[:n_val])
train_ids = set(img_ids[n_val:])

def subset_coco(data, keep_ids):
    return {
        "images": [im for im in data["images"] if im["id"] in keep_ids],
        "annotations": [a for a in data["annotations"] if a["image_id"] in keep_ids],
        "categories": data["categories"],
        "licenses": data.get("licenses", []),
        "info": data.get("info", {})
    }

train_json = subset_coco(coco, train_ids)
val_json   = subset_coco(coco, val_ids)

TRAIN_JSON = f"{DATA_ROOT}/train_coco.json"
VAL_JSON   = f"{DATA_ROOT}/val_coco.json"
json.dump(train_json, open(TRAIN_JSON, "w"))
json.dump(val_json,   open(VAL_JSON, "w"))

print("Train images:", len(train_json["images"]), "Val images:", len(val_json["images"]))


Train images: 56 Val images: 14


In [ ]:
import os, shutil, json, cv2, numpy as np
from pycocotools import mask as maskUtils

# YOLO dir layout
for split in ["train", "val"]:
    os.makedirs(f"{DATA_ROOT}/images/{split}", exist_ok=True)
    os.makedirs(f"{DATA_ROOT}/labels/{split}", exist_ok=True)

def polygons_from_rle(rle_mask):
    """Decode RLE -> contours (list of Nx2 arrays)."""
    m = maskUtils.decode(rle_mask).astype(np.uint8)  # HxW
    cnts, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polys = []
    for c in cnts:
        if len(c) < 3:
            continue
        c = c.squeeze(1)  # Nx2
        polys.append(c)
    return polys

def write_yolo_seg_txt(txt_path, instances, W, H):
    """instances: list of polygons (each as Nx2) with class id 0."""
    lines = []
    for poly in instances:
        if poly.shape[0] < 3:
            continue
        # normalize to [0,1]
        x = np.clip(poly[:,0] / W, 0, 1)
        y = np.clip(poly[:,1] / H, 0, 1)
        # YOLO line: <cls> x1 y1 x2 y2 ...
        coords = " ".join([f"{a:.6f} {b:.6f}" for a,b in zip(x, y)])
        lines.append(f"0 {coords}")
    if lines:
        with open(txt_path, "w") as f:
            f.write("\n".join(lines))
    else:
        # If no instances, create empty file to avoid dataloader crashes
        open(txt_path, "w").close()

def convert_split(split_json_path, split):
    data = json.load(open(split_json_path))
    anns_by_img = {}
    for a in data["annotations"]:
        anns_by_img.setdefault(a["image_id"], []).append(a)

    for im in data["images"]:
        img_id   = im["id"]
        file_name= im["file_name"]
        W, H     = im["width"], im["height"]

        src = os.path.join(IMAGES_DIR, file_name)
        dst = os.path.join(DATA_ROOT, "images", split, file_name)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy2(src, dst)

        polys_all = []
        for a in anns_by_img.get(img_id, []):
            # (Optional) if you later add more classes, map to 0-based index here.
            seg = a.get("segmentation", None)
            if seg is None:
                # fall back: build polygon from bbox (very coarse)
                x,y,w,h = a["bbox"]
                poly = np.array([[x,y],[x+w,y],[x+w,y+h],[x,y+h]], dtype=np.float32)
                polys_all.append(poly)
                continue

            # COCO segmentation can be polygon (list of lists) or RLE (dict)
            if isinstance(seg, list):
                # multiple polygons possible for one instance
                for s in seg:
                    if len(s) < 6:  # need at least 3 points
                        continue
                    arr = np.array(s, dtype=np.float32).reshape(-1,2)
                    polys_all.append(arr)
            elif isinstance(seg, dict) and {"size","counts"} <= set(seg.keys()):
                for poly in polygons_from_rle(seg):
                    polys_all.append(poly.astype(np.float32))
            else:
                # unknown format; skip this instance gracefully
                continue

        label_path = os.path.join(DATA_ROOT, "labels", split, os.path.splitext(file_name)[0] + ".txt")
        write_yolo_seg_txt(label_path, polys_all, W, H)

convert_split(TRAIN_JSON, "train")
convert_split(VAL_JSON,   "val")

print("Done converting. Example label file:")
sample_lbls = os.listdir(f"{DATA_ROOT}/labels/train")[:3]
print(sample_lbls)
print(open(os.path.join(f"{DATA_ROOT}/labels/train", sample_lbls[0])).read()[:200])


Done converting. Example label file:
['73611d76-bin4.txt', '4191e48f-bin7.txt', '69d671f5-bin10.txt']
0 0.323367 0.241206 0.159799 0.260302 0.161307 0.275377 0.171859 0.279397 0.186181 0.417085 0.198995 0.437186 0.206533 0.522613 0.238945 0.787940 0.242714 0.827136 0.266080 0.855276 0.293216 0.862312 


In [ ]:
DATA_YAML = f"{DATA_ROOT}/data.yaml"
open(DATA_YAML, "w").write(f"""# YOLOv8 Segmentation dataset
path: {DATA_ROOT}
train: images/train
val: images/val
names: ["Trash"]  # one class
""")
print("Wrote:", DATA_YAML)


Wrote: /content/drive/MyDrive/Archive/yolo/data.yaml


In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8s-seg.pt")  # try yolov8m-seg.pt later

results = model.train(
    data=DATA_YAML,
    epochs=60,
    imgsz=896,
    batch=8,
    lr0=1e-3,
    cache="ram",
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=5, translate=0.05, scale=0.1, shear=2.0,
    fliplr=0.5,
    mosaic=0.5,
    copy_paste=0.2,
    project=f"{BASE}/runs",
    name="yolov8s_seg_trash"
)


New https://pypi.org/project/ultralytics/8.3.203 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.29 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/drive/MyDrive/Archive/yolo/data.yaml, epochs=60, time=None, patience=100, batch=8, imgsz=896, save=True, save_period=-1, cache=ram, device=None, workers=8, project=/content/drive/MyDrive/Archive/runs, name=yolov8s_seg_trash2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, clas

train: Scanning /content/drive/MyDrive/Archive/yolo/labels/train... 56 images, 0 backgrounds, 0 corrupt: 100%|██████████| 56/56 [00:00<00:00, 73.82it/s]

train: New cache created: /content/drive/MyDrive/Archive/yolo/labels/train.cache


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 56/56 [00:02<00:00, 20.43it/s]


...

      60/60      4.82G     0.4445     0.6983     0.3881     0.8445         25        896: 100%|██████████| 7/7 [00:02<00:00,  3.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.54it/s]

                   all         14         43      0.484      0.349      0.347      0.217      0.529      0.393      0.364      0.232



60 epochs completed in 0.071 hours.
Optimizer stripped from /content/drive/MyDrive/Archive/runs/yolov8s_seg_trash2/weights/last.pt, 23.9MB
Optimizer stripped from /content/drive/MyDrive/Archive/runs/yolov8s_seg_trash2/weights/best.pt, 23.9MB

Validating /content/drive/MyDrive/Archive/runs/yolov8s_seg_trash2/weights/best.pt...
Ultralytics 8.3.29 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,779,987 parameters, 0 gradients, 39.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.33it/s]


                   all         14         43      0.489      0.356      0.348      0.217       0.53      0.393      0.366      0.232
Speed: 0.4ms preprocess, 10.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/Archive/runs/yolov8s_seg_trash2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
metrics = model.val(data=DATA_YAML, imgsz=896, iou=0.50, conf=0.001, plots=True)
print(metrics)


Ultralytics 8.3.29 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,779,987 parameters, 0 gradients, 39.9 GFLOPs


val: Scanning /content/drive/MyDrive/Archive/yolo/labels/val.cache... 14 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14/14 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.0GB RAM): 100%|██████████| 14/14 [00:00<00:00, 25.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.14it/s]


                   all         14         43      0.654      0.326      0.371      0.228      0.547      0.395      0.402      0.247
Speed: 0.7ms preprocess, 39.9ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/drive/MyDrive/Archive/runs/yolov8s_seg_trash22
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f709c9e4860>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     

In [ ]:
import os
val_imgs = [os.path.join(DATA_ROOT, "images/val", f) for f in os.listdir(f"{DATA_ROOT}/images/val")[:8]]
pred = model.predict(source=val_imgs, imgsz=1024, conf=0.25, save=True,
                     project=f"{BASE}/runs", name="pred_std", line_width=2)
print("Saved to:", pred[0].save_dir if pred else "n/a")



0: 1024x1024 (no detections), 40.0ms
1: 1024x1024 5 Trashs, 40.0ms
2: 1024x1024 1 Trash, 40.0ms
3: 1024x1024 2 Trashs, 40.0ms
4: 1024x1024 2 Trashs, 40.0ms
5: 1024x1024 2 Trashs, 40.0ms
6: 1024x1024 1 Trash, 40.0ms
7: 1024x1024 1 Trash, 40.0ms
Speed: 8.8ms preprocess, 40.0ms inference, 4.1ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to /content/drive/MyDrive/Archive/runs/pred_std
Saved to: /content/drive/MyDrive/Archive/runs/pred_std
